#### In order to make our gunshots dataset large we gathered gunshots audios from different datasets.Following code is writtrn to extract gunshot audios from MIVIA dataset. In this dataset there were three categories of audios (gunshot, glass break and screams) embedde in a single audio. Timing information for each audio was provided in xml file such that each class category start and end time was given in xml file. So gun shots were extracted from each audio using xml file timing information.

#### importing libraries

In [ ]:
import xml.etree.ElementTree as ET
import sys
import numpy as np
import wave
import pyaudio
import struct
import math
import os

#### complete code for gunshot,scream,glass break,background audio extraction from single audio of MIVIA dataset

In [ ]:
#this training audio path can be changed to testing for extracting testing audios
#MIVIA DATASET Training sound path
File="E:\\All Data\\study\\MS\\2\\machine learning\\Project\\MIVIA_dataset\\MIVIA_DB4_dist\\training\\sounds"    
#MIVIA DATASET Training xml path
path="E:\\All Data\\study\\MS\\2\\machine learning\\Project\\MIVIA_dataset\\MIVIA_DB4_dist\\training"
folder=path
#loop over entire xml files
for xml in os.listdir(path):  
    #check if xml
    if xml.split('.')[-1]=="xml":
        #load xml file
       tree = ET.parse(path+"\\"+xml)
       #get root of xml file
       root = tree.getroot()
       #initialize counters for naming
       glass=0
       gun=0
       scream=0
       back=0
    else:
        continue
#loop over xml root->child         
    for i in root:
        for j in i:
            for k in j:
                #get the audio class name
                if k.tag=="CLASS_NAME":
                    name=k.text.split("/")[0]
                #get start second of that audio class from xml and round the value to floor   
                if k.tag=="STARTSECOND":
                    start = int(math.floor(float(k.text.split("/")[0])))
                #get end second of that audio class from xml and round the value to ceil
                if k.tag=="ENDSECOND":
                    end = int(math.ceil(float(k.text.split("/")[0])))
            #check if folder with specific class name exists or not if not then create
            if os.path.isdir(folder+"\\"+"train_"+name):
                pass
            else:
                #os.rmdir(folder+"\\"+name)
                os.mkdir(folder+"\\"+"train_"+name)
            print (start,end,name)
            #print store
            length =end-start
            #loop over MIVIA sound folder and get the storing audio class path
            for audio in os.listdir(File):
                chunk = 1024
                #check if audio name matches xml to be processed 
                if audio.split('.')[0].split('_')[0]==xml.split('.')[0]:
                    #check the class name of audio section to be extracted and give storing path acordingly
                    if name=="glass":
                        store=folder+"\\"+"train_"+name+"\\"+name+str(glass)+"_"+audio.split('.')[0]+".wav"
                        glass=glass+1
                    if name=="gunshots":
                        store=folder+"\\"+"train_"+name+"\\"+name+str(gun)+"_"+audio.split('.')[0]+".wav"
                        gun=gun+1
                    if name=="screams":
                        store=folder+"\\"+"train_"+name+"\\"+name+str(scream)+"_"+audio.split('.')[0]+".wav"
                        scream=scream+1
                    if name=="background":
                        store=folder+"\\"+"train_"+name+"\\"+name+str(back)+"_"+audio.split('.')[0]+".wav"
                        back=back+1
                    #open audio file
                    spf = wave.open(File+"\\"+audio, 'rb')
                    signal = spf.readframes(-1)
                    #frame=signal
                    signal = np.fromstring(signal, 'Int16')
                    p = pyaudio.PyAudio()
                    stream = p.open(format =
                                    p.get_format_from_width(spf.getsampwidth()),
                                    channels = spf.getnchannels(),
                                    rate = spf.getframerate(),
                                    output = True)
                    #extract the audio of specific length of specific class{glass break,gunshot,background or scream} provided in xml 
                    pos=spf.getframerate()*length
                    signal =signal[start*spf.getframerate():(start*spf.getframerate()) + pos]
                    sig=signal[1:chunk]
                    inc = 0;
                    data=0;
                    frame=[]
                    #get the audio extracted data again to be stored
                    while data != '':
                        data = struct.pack("%dh"%(len(sig)), *list(sig))
                        frame.append(data)
                        #stream.write(data)
                        inc=inc+chunk
                        sig=signal[inc:inc+chunk]
                    stream.close()
                    p.terminate()
                    wf = wave.open(store, 'wb') 
                    wf.setnchannels(spf.getnchannels())
                    #print (spf.getnchannels())
                    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
                    #print (spf.getframerate())
                    wf.setframerate(spf.getframerate())
                    wf.writeframes(b''.join(frame))# store the extracted audio
                    wf.close()
                else:
                    pass
